<a href="https://colab.research.google.com/github/yasseriz/SRGAN/blob/yasseriz-final/Autoencoder_trial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Cloned SRGAN repo
!git clone https://github.com/yasseriz/SRGAN.git

Cloning into 'SRGAN'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 64 (delta 4), reused 0 (delta 0), pack-reused 52
Unpacking objects: 100% (64/64), done.


In [0]:
cd SRGAN/

/content/SRGAN


In [0]:
!git checkout yasseriz-final

Branch 'yasseriz-final' set up to track remote branch 'yasseriz-final' from 'origin'.
Switched to a new branch 'yasseriz-final'


In [0]:
# !cd ..
%ls


baboon.png   coastguard.png  flowers.png  man.png      ppt3.png      zebra.png
barbara.png  comic.png       foreman.png  monarch.png  sample_data/
bridge.png   face.png        lenna.png    pepper.png   SRGAN/


In [0]:
!pip install scipy==1.0.0

     |████████████████████████████████| 50.0MB 84kB/s 
ERROR: tensorflow 2.2.0rc2 has requirement scipy==1.4.1; python_version >= "3", but you'll have scipy 1.0.0 which is incompatible.
ERROR: seaborn 0.10.0 has requirement scipy>=1.0.1, but you'll have scipy 1.0.0 which is incompatible.
ERROR: plotnine 0.6.0 has requirement scipy>=1.2.0, but you'll have scipy 1.0.0 which is incompatible.
ERROR: cvxpy 1.0.28 has requirement scipy>=1.1.0, but you'll have scipy 1.0.0 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1


In [0]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [0]:
import keras
import numpy as np
from scipy.misc import imresize, imread
from keras import Input
from keras.layers import BatchNormalization, Activation, Add, LeakyReLU, Dense, MaxPooling2D
from keras.layers.advanced_activations import PReLU
from keras.layers.convolutional import Conv2D, UpSampling2D, Conv2DTranspose
from keras.models import Model
from keras.applications import VGG19
import matplotlib.pyplot as plt
from glob import glob
import scipy.misc
from PIL import Image


In [0]:
def sample_images(data_dir, batch_size, high_resolution_shape, low_resolution_shape):

    # Make a list of all images inside the data directory
    all_images = data_dir

    # Choose a random batch of images
    images_batch = np.random.choice(all_images, size=batch_size)

    low_resolution_images = []
    high_resolution_images = []

    for img in images_batch:
        # Get an ndarray of the current image
        img1 = imread(img, mode='RGB')
        img1 = img1.astype(np.float32)

        # Resize the image
        img1_high_resolution = imresize(img1, high_resolution_shape)
        img1_low_resolution = imresize(img1, low_resolution_shape)

        # Do a random horizontal flip
        if np.random.random() < 0.5:
          img1_high_resolution = np.fliplr(img1_high_resolution)
          img1_low_resolution = np.fliplr(img1_low_resolution)

        high_resolution_images.append(img1_high_resolution)
        low_resolution_images.append(img1_low_resolution)

    # Convert the lists to Numpy NDArrays
    return np.array(high_resolution_images), np.array(low_resolution_images)

In [0]:
def save_images(low_resolution_images, high_resolution_images, generated_images, path):
    fig = plt.figure()
    ax = fig.add_subplot(1, 3, 1)
    ax.imshow(low_resolution_images[0])
    ax.axis("off")
    ax.set_title("Low-resolution")

    ax = fig.add_subplot(1, 3, 2)
    ax.imshow(high_resolution_images[0])
    ax.axis("off")
    ax.set_title("Original")

    ax = fig.add_subplot(1, 3, 3)
    ax.imshow(generated_images[0])
    ax.axis("off")
    ax.set_title("Generated")

    plt.savefig(path)
    plt.clf()

In [0]:
# AutoEncoder Block
def build_autoencoder():

    input_shape = (256, 256, 3)
    input_layer = Input(shape=input_shape)


    x = Conv2D(512, (3, 3), activation='relu', strides=1, padding='same')(input_layer)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(256, (3, 3), activation='relu', strides=1, padding='same')(x)
    encoded = MaxPooling2D((2, 2), padding='same')(x)

    # at this point the representation is (7, 7, 32)

    x = Conv2D(256, (3, 3), activation='relu', strides=1, padding='same')(encoded)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(512, (3, 3), activation='relu', strides=1, padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    decoded = Conv2D(3, (3, 3), activation='tanh', strides=1, padding='same')(x)

    autoencoder = Model(input_layer, decoded)
    return autoencoder
    autoencoder.compile(optimizer='adadelta', loss='mse')

In [0]:
# Using a pretrained network
def build_vgg():
    """
    Builds a pre-trained VGG19 model that outputs image features extracted at the
    third block of the model
    """
    input_shape = (64, 64, 3)
    vgg = VGG19(weights="imagenet", input_shape=input_shape, include_top=False)
    # Set the outputs to outputs of last conv. layer in block 3
    # See architecture at: https://github.com/keras-team/keras/blob/master/keras/applications/vgg19.py
    newOutput = vgg.predict(low_resolution_images)
    print(vgg.summary())
    img = Input(shape=input_shape)

    # Extract the image features
    img_features = vgg(img)

    return Model(inputs=[img], outputs=[img_features], name='VGG')

In [0]:
data_dir = glob('./Dataset/*')
epochs = 501
batch_size = 1
low_resolution_shape=(64, 64, 3)
high_resolution_shape=(256, 256, 3)


In [0]:
autoencoder = build_autoencoder()
autoencoder.compile(optimizer='Adam', loss='mse')
# print(autoencoder.metric_names)
print(autoencoder.summary())


Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 256, 256, 512)     14336     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 128, 128, 512)     0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 128, 128, 256)     1179904   
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 64, 64, 256)       0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 64, 64, 256)       590080    
_________________________________________________________________
up_sampling2d_5 (UpSampling2 (None, 128, 128, 256)     0   

In [0]:
vgg = build_vgg()
vgg.trainable = False
vgg.compile(optimizer='Adam', metrics=['accuracy'], loss='mse')
print(vgg.summary())

In [0]:
# Adding black pixels to image to simulate generated image from generator
newpath = 'NoisyData/'
datapoints = 16384
xpoints = np.random.randint(0, 256)
for j in range(14):
    high_resolution_images, low_resolution_images = sample_images(data_dir=data_dir, batch_size=batch_size,
                                                                            high_resolution_shape=high_resolution_shape,
                                                                            low_resolution_shape=low_resolution_shape)
    # high_resolution_images = high_resolution_images / 127.5 - 1
    for i in range(datapoints):
        xpoints = np.random.randint(0, 256)
        ypoints = np.random.randint(0, 256)
        zpoints = np.random.randint(0, 3)
        high_resolution_images[0][xpoints][ypoints][0] = 0
        high_resolution_images[0][xpoints][ypoints][1] = 0
        high_resolution_images[0][xpoints][ypoints][2] = 0
    new_noisy_high_resolution_images = high_resolution_images
    print(new_noisy_high_resolution_images.shape)
    # fig = plt.figure()
    # plt.imshow(high_resolution_images[0])
    # plt.savefig(newpath+str(j))
    # plt.clf()
    scipy.misc.imsave(newpath+str(j)+'.png', new_noisy_high_resolution_images[0])
    # im = Image.fromarray(new_noisy_high_resolution_images[0])
    # im.save(newpath+str(j)+'.png')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


(1, 256, 256, 3)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


(1, 256, 256, 3)
(1, 256, 256, 3)
(1, 256, 256, 3)
(1, 256, 256, 3)
(1, 256, 256, 3)
(1, 256, 256, 3)
(1, 256, 256, 3)
(1, 256, 256, 3)
(1, 256, 256, 3)
(1, 256, 256, 3)
(1, 256, 256, 3)
(1, 256, 256, 3)
(1, 256, 256, 3)


In [0]:
 for epoch in range(epochs):
    high_resolution_images, low_resolution_images = sample_images(data_dir=data_dir, batch_size=batch_size,
                                                                            high_resolution_shape=high_resolution_shape,
                                                                            low_resolution_shape=low_resolution_shape)
    high_resolution_images = high_resolution_images / 127.5 - 1
    low_resolution_images = low_resolution_images / 127.5 - 1
    noise_factor = 0.2
    noisy_low_resolution_images = high_resolution_images + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=high_resolution_shape) 
    # noisy_low_resolution_images = np.clip(noisy_low_resolution_images, 0., 1.)
    
    loss = autoencoder.train_on_batch(noisy_low_resolution_images, high_resolution_images)
    print(loss)
    if epoch%10==0:
        high_resolution_images, low_resolution_images = sample_images(data_dir=data_dir, batch_size=batch_size,
                                                                              high_resolution_shape=high_resolution_shape,
                                                                              low_resolution_shape=low_resolution_shape)
        # Normalizing the images
        high_resolution_images = high_resolution_images / 127.5 - 1
        low_resolution_images = low_resolution_images / 127.5 - 1
        noisy_low_resolution_images = high_resolution_images + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=high_resolution_shape) 
        # noisy_low_resolution_images = np.clip(noisy_low_resolution_images, 0., 1.)
        generated_images = autoencoder.predict_on_batch(noisy_low_resolution_images)

        high_resolution_images = 0.5 * high_resolution_images + 0.5
        low_resolution_images = 0.5 * low_resolution_images + 0.5
        generated_images = 0.5 * generated_images + 0.5
        print(generated_images.shape)
        for index, img in enumerate(generated_images):
            save_images(noisy_low_resolution_images, high_resolution_images, generated_images,
                        path="Results/img_{}".format(epoch))


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


0.24094364


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


(1, 256, 256, 3)
0.37607557
0.7706921
1.4117682
1.1673467
2.2498035
0.98917407
0.8314586
0.5385311
0.9068308
0.5351839


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


...

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>